## 循环神经网络 — 从0开始
前面的教程里我们使用的网络都属于前馈神经网络。之所以叫前馈，是因为整个网络是一条链，每一层的结果都是反馈给下一层。这一节我们介绍循环神经网络，这里每一层不仅输出给下一层，同时还输出一个隐含状态，给当前层在处理下一个样本时使用。下图展示这两种网络的区别。
![image.png](http://zh.gluon.ai/_images/rnn_1.png)

循环神经网络的这种结构使得它适合处理前后有依赖关系数据样本。我们拿语言模型举个例子来解释这个是怎么工作的。语言模型的任务是给定句子的前$t$个字符，然后预测第$t+1$个字符。假设我们的句子是“你好世界”，使用前馈神经网络来预测的一个做法是，在时间1输入“你”，预测”好“，时间2向同一个网络输入“好”预测“世”。下图左边展示了这个过程。

![image.png](http://zh.gluon.ai/_images/rnn_2.png)

注意到一个问题是，当我们预测“世”的时候只给了“好”这个输入，而完全忽略了“你”。直觉上“你”这个词应该对这次的预测比较重要。虽然这个问题通常可以通过n-gram来缓解，就是说预测第$t+1$个字符的时候，我们输入前$n$个字符。如果$n=1$，那就是我们这里用的。我们可以增大$n$来使得输入含有更多信息。但我们不能任意增大$n$，因为这样通常带来模型复杂度的增加从而导致需要大量数据和计算来训练模型。

循环神经网络使用一个隐含状态来记录前面看到的数据来帮助当前预测。上图右边展示了这个过程。在预测“好”的时候，我们输出一个隐含状态。我们用这个状态和新的输入“好”来一起预测“世”，然后同时输出一个更新过的隐含状态。我们希望前面的信息能够保存在这个隐含状态里，从而提升预测效果。

### 循环神经网络
在对输入输出数据有了解后，我们来正式介绍循环神经网络。

首先回忆一下单隐含层的前馈神经网络的定义，例如多层感知机。假设隐含层的激活函数是$\phi$，对于一个样本数为$n$特征向量维度为$x$的批量数据$\mathbf{X} \in \mathbb{R}^{n \times x}$（$X$是一个$n$行$x$列的实数矩阵）来说，那么这个隐含层的输出就是

$H=ϕ(XWxh+bh)$

假定隐含层长度为$h$，其中的$\mathbf{W}_{xh} \in \mathbb{R}^{x \times h}$是权重参数。偏移参数 $\mathbf{b}_h \in \mathbb{R}^{1 \times h}$在与前一项$\mathbf{X} \mathbf{W}_{xh} \in \mathbb{R}^{n \times h}$ 相加时使用了广播。这个隐含层的输出的尺寸为$\mathbf{H} \in \mathbb{R}^{n \times h}$。

把隐含层的输出$\mathbf{H}$作为输出层的输入，最终的输出

$\hat{\mathbf{Y}} = \text{softmax}(\mathbf{H} \mathbf{W}_{hy} + \mathbf{b}_y)$

假定每个样本对应的输出向量维度为y，其中$\hat{\mathbf{Y}} \in \mathbb{R}^{n \times y}, \mathbf{W}_{hy} \in \mathbb{R}^{h \times y}, \mathbf{b}_y \in \mathbb{R}^{1 \times y}$且两项相加使用了广播。

将上面网络改成循环神经网络，我们首先对输入输出加上时间戳$t$。假设$\mathbf{X}_t \in \mathbb{R}^{n \times x}$是序列中的第$t$个批量输入（样本数为$n$，每个样本的特征向量维度为$x$），对应的隐含层输出是隐含状态$\mathbf{H}_t \in \mathbb{R}^{n \times h}$（隐含层长度为$h$），而对应的最终输出是$\hat{\mathbf{Y}}_t \in \mathbb{R}^{n \times y}$（每个样本对应的输出向量维度为$y$）。在计算隐含层的输出的时候，循环神经网络只需要在前馈神经网络基础上加上跟前一时间$t−1$输入隐含层$\mathbf{H}_{t-1} \in \mathbb{R}^{n \times h}$的加权和。为此，我们引入一个新的可学习的权重$\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$：

$\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h)$

输出的计算跟前面一致：

$\hat{\mathbf{Y}}_t = \text{softmax}(\mathbf{H}_t \mathbf{W}_{hy}  + \mathbf{b}_y)$

一开始我们提到过，隐含状态可以认为是这个网络的记忆。该网络中，时刻$t$的隐含状态就是该时刻的隐含层变量$\mathbf{H}_t$。它存储前面时间里面的信息。我们的输出是只基于这个状态。最开始的隐含状态里的元素通常会被初始化为0。

### 周杰伦歌词数据集
为了实现并展示循环神经网络，我们使用周杰伦歌词数据集来训练模型作词。该数据集里包含了著名创作型歌手周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》所有歌曲的歌词。

![image.png](http://zh.gluon.ai/_images/jay.jpg)

下面我们读取这个数据并看看前面49个字符（char）是什么样的：



In [1]:
import zipfile
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip', 'r') as zin:
    zin.extractall('../../data/')

with open('../../data/jaychou_lyrics.txt') as f:
    corpus_chars = f.read().decode('utf-8')
print(corpus_chars)

想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你
这样的甜蜜
让我开始乡相信命运
感谢地心引力
让我碰到你
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你
这样的甜蜜
让我开始乡相信命运
感谢地心引力
让我碰到你
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
漂亮的让我面红的可爱女人
温柔的让我心疼的可爱女人
透明的让我感动的可爱女人
坏坏的让我疯狂的可爱女人
坏坏的让我疯狂的可爱女人
如果说怀疑 可以造句如果说分离 能够翻译
如果这一切 真的可以
我想要将我的寂寞封闭
然后在这里 不限日期
然后将过去 慢慢温习
让我爱上你 那场悲剧
是你完美演出的一场戏
宁愿心碎哭泣 再狠狠忘记 你爱过我的证据
让晶莹的泪滴 闪烁成回忆 伤人的美丽
你的完美主义 太彻底
让我连恨都难以下笔
将真心抽离写成日记 像是一场默剧
你的完美主义 太彻底
分手的话像语言暴力
我已无能为力再提起 决定中断熟悉
然后在这里 不限日期
然后将过去 慢慢温习
让我爱上你 那场悲剧
是你完美演出的一场戏
宁愿心碎哭泣 再狠狠忘记 你爱过我的证据
让晶莹的泪滴 闪烁成回忆 伤人的美丽
你的完美主义 太彻底
让我连恨都难以下笔
将真心抽离写成日记 像是一场默剧
你的完美主义 太彻底
分手的话像语言暴力
我已无能为力再提起 决定中断熟悉
周杰伦 周杰伦
一步两步三步四步望著天 看星星
一颗两颗三颗四颗 连成线一步两步三步四步望著天 看星星
一颗两颗三颗四颗 连成线乘著风 游荡在蓝天边
一片云掉落在我面前
捏成你的形状

我们看一下数据集里的字符数。



In [2]:
len(corpus_chars)


64925

接着我们稍微处理下数据集。为了打印方便，我们把换行符替换成空格，然后截去后面一段使得接下来的训练会快一点。



In [3]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
print corpus_chars[0]


想


### 字符的数值表示
先把数据里面所有不同的字符拿出来做成一个字典：



In [4]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])

vocab_size = len(char_to_idx)

print('vocab size:', vocab_size)

('vocab size:', 2613)


然后可以把每个字符转成从0开始的索引(index)来方便之后的使用。



In [5]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]

sample = corpus_indices[:40]

print('chars: \n', ''.join([idx_to_char[idx] for idx in sample]))
print('\nindices: \n', sample)

('chars: \n', u'\u60f3\u8981\u6709\u76f4\u5347\u673a \u60f3\u8981\u548c\u4f60\u98de\u5230\u5b87\u5b99\u53bb \u60f3\u8981\u548c\u4f60\u878d\u5316\u5728\u4e00\u8d77 \u878d\u5316\u5728\u5b87\u5b99\u91cc \u6211\u6bcf\u5929\u6bcf\u5929\u6bcf')
('\nindices: \n', [995, 651, 852, 2006, 2076, 1287, 1279, 995, 651, 525, 2542, 1729, 2110, 1360, 2443, 910, 1279, 995, 651, 525, 2542, 1600, 1448, 903, 1123, 2608, 1279, 1600, 1448, 903, 1360, 2443, 321, 1279, 2228, 2374, 393, 2374, 393, 2374])


### 时序数据的批量采样
同之前一样我们需要每次随机读取一些（`batch_size`个）样本和其对用的标号。这里的样本跟前面有点不一样，这里一个样本通常包含一系列连续的字符（前馈神经网络里可能每个字符作为一个样本）。

如果我们把序列长度（`num_steps`）设成5，那么一个可能的样本是“想要有直升”。其对应的标号仍然是长为5的序列，每个字符是对应的样本里字符的后面那个。例如前面样本的标号就是“要有直升机”。

#### 随机批量采样
下面代码每次从数据里随机采样一个批量。

In [6]:
import random
import numpy as np

def data_iter_random(corpus_indices, batch_size, num_steps):
    # 减一是因为label的索引是相应data的索引加一
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    # 随机化样本
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回num_steps个数据
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]

    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        data = np.array(
            [_data(j * num_steps) for j in batch_indices])
        label = np.array(
            [_data(j * num_steps + 1) for j in batch_indices])
        yield data.astype(np.float32), label

为了便于理解时序数据上的随机批量采样，让我们输入一个从0到29的人工序列，看下读出来长什么样：



In [7]:
my_seq = list(range(30))

for data, label in data_iter_random(my_seq, batch_size=2, num_steps=3):
    print('data: ', data, '\nlabel:', label, '\n')

('data: ', array([[12., 13., 14.],
       [ 3.,  4.,  5.]], dtype=float32), '\nlabel:', array([[13, 14, 15],
       [ 4,  5,  6]]), '\n')
('data: ', array([[ 6.,  7.,  8.],
       [24., 25., 26.]], dtype=float32), '\nlabel:', array([[ 7,  8,  9],
       [25, 26, 27]]), '\n')
('data: ', array([[ 0.,  1.,  2.],
       [18., 19., 20.]], dtype=float32), '\nlabel:', array([[ 1,  2,  3],
       [19, 20, 21]]), '\n')
('data: ', array([[15., 16., 17.],
       [ 9., 10., 11.]], dtype=float32), '\nlabel:', array([[16, 17, 18],
       [10, 11, 12]]), '\n')


由于各个采样在原始序列上的位置是随机的时序长度为num_steps的连续数据点，相邻的两个随机批量在原始序列上的位置不一定相毗邻。因此，在训练模型时，读取每个随机时序批量前需要重新初始化隐含状态。

### 相邻批量采样
除了对原序列做随机批量采样之外，我们还可以使相邻的两个随机批量在原始序列上的位置相毗邻。

In [8]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = np.array((corpus_indices))
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size

    indices = corpus_indices[0: batch_size * batch_len].reshape((
        batch_size, batch_len))
    # 减一是因为label的索引是相应data的索引加一
    epoch_size = (batch_len - 1) // num_steps

    for i in range(epoch_size):
        i = i * num_steps
        data = indices[:, i: i + num_steps]
        label = indices[:, i + 1: i + num_steps + 1]
        yield data, label

相同地，为了便于理解时序数据上的相邻批量采样，让我们输入一个从0到29的人工序列，看下读出来长什么样

In [9]:
my_seq = list(range(30))

for data, label in data_iter_consecutive(my_seq, batch_size=2, num_steps=3):
    print('data: ', data, '\nlabel:', label, '\n')

('data: ', array([[ 0,  1,  2],
       [15, 16, 17]]), '\nlabel:', array([[ 1,  2,  3],
       [16, 17, 18]]), '\n')
('data: ', array([[ 3,  4,  5],
       [18, 19, 20]]), '\nlabel:', array([[ 4,  5,  6],
       [19, 20, 21]]), '\n')
('data: ', array([[ 6,  7,  8],
       [21, 22, 23]]), '\nlabel:', array([[ 7,  8,  9],
       [22, 23, 24]]), '\n')
('data: ', array([[ 9, 10, 11],
       [24, 25, 26]]), '\nlabel:', array([[10, 11, 12],
       [25, 26, 27]]), '\n')


### One-hot向量
注意到每个字符现在是用一个整数来表示，而输入进网络我们需要一个定长的向量。一个常用的办法是使用one-hot来将其表示成向量。也就是说，如果一个字符的整数值是i, 那么我们创建一个全0的长为vocab_size的向量，并将其第i位设成1。该向量就是对原字符的one-hot向量。

In [10]:
import numpy as np

def one_hot(positions, vocal_size):
    if len(positions.shape) == 0:
        positions = np.expand_dims(positions, 0)
        positions = np.expand_dims(positions, 1)
    if len(positions.shape) == 1:
        positions = np.expand_dims(positions, 1)
    all_zeros = np.zeros((positions.shape[0], vocab_size))
    for i in xrange(positions.shape[0]):
        for j in xrange(positions.shape[1]):
            all_zeros[i, int(positions[i, j])] = 1
        
    return all_zeros
        
one_hot(np.array([0, 2]), vocab_size)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

记得前面我们每次得到的数据是一个`batch_size * num_steps`的批量。下面这个函数将其转换成`num_steps`个可以输入进网络的`batch_size * vocab_size`的矩阵。对于一个长度为`num_steps`的序列，每个批量输入$\mathbf{X} \in \mathbb{R}^{n \times x}$，其中`n= batch_size`，而`x=vocab_size`（onehot编码向量维度）。



In [11]:
def get_inputs(data):
    return [one_hot(X, vocab_size) for X in data.T]

print data
inputs = get_inputs(data)

print inputs
print('input length: ', len(inputs))
print('input shape: ', inputs[0].shape)

[[ 9 10 11]
 [24 25 26]]
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
('input length: ', 3)
('input shape: ', (2, 2613))


### 初始化模型参数
对于序列中任意一个时间戳，一个字符的输入是维度为`vocab_size`的`one-hot`向量，对应输出是预测下一个时间戳为词典中任意字符的概率，因而该输出是维度为`vocab_size`的向量。

当序列中某一个时间戳的输入为一个样本数为`batch_size`（对应模型定义中的$n$）的批量，每个时间戳上的输入和输出皆为尺寸`batch_size * vocab_size`（对应模型定义中的$n×x$）的矩阵。假设每个样本对应的隐含状态的长度为`hidden_dim`（对应模型定义中隐含层长度$h$），根据矩阵乘法定义，我们可以推断出模型隐含层和输出层中各个参数的尺寸。



In [12]:
import tensorflow as tf

input_dim = vocab_size
# 隐含状态长度
hidden_dim = 256
output_dim = vocab_size
weight_scale = .01


with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):
    W_xh = tf.get_variable(name='weights_hidden', shape=[input_dim, hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_h = tf.get_variable(name='bias_hidden', shape=[hidden_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)

    W_hh = tf.get_variable(name='times_hidden2', shape=[hidden_dim,hidden_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)

    W_hy = tf.get_variable(name='weights_output', shape=[hidden_dim, output_dim], initializer=tf.random_normal_initializer(mean=0.0, stddev=weight_scale), dtype=tf.float32)
    b_y = tf.get_variable(name='bias_output', shape=[output_dim], initializer=tf.constant_initializer(0.0), dtype=tf.float32)


params = [W_xh, b_h, W_hh, W_hy, b_y]


### 定义模型
当序列中某一个时间戳的输入为一个样本数为`batch_size`的批量，而整个序列长度为`num_steps`时，以下`rnn`函数的`inputs`和`outputs`皆为`num_steps 个尺寸为batch_size * vocab_size`的矩阵，隐含变量$H$是一个尺寸为`batch_size * hidden_dim`的矩阵。该隐含变量$H$也是循环神经网络的隐含状态`state`。

我们将前面的模型公式翻译成代码。这里的激活函数使用了按元素操作的双曲正切函数

$\text{tanh}(x) = \frac{1 - e^{-2x}}{1 + e^{-2x}}$

需要注意的是，双曲正切函数的值域是$[−1,1]$。如果自变量均匀分布在整个实域，该激活函数输出的均值为0。



In [27]:
def rnn(inputs, state, params):
    # inputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵。
    # H: 尺寸为 batch_size * hidden_dim 矩阵。
    # outputs: num_steps 个尺寸为 batch_size * vocab_size 矩阵。
    H = state
    W_xh, b_h, W_hh, W_hy, b_y = params
    outputs = []

    num_steps = inputs.get_shape().as_list()[0]
    for i in range(num_steps):
        X = inputs[i]
        H = tf.nn.tanh(tf.matmul(X, W_xh) + tf.matmul(H, W_hh) + b_h)
        Y = tf.matmul(H, W_hy) + b_y
        outputs.append(Y)
    return (outputs, H)


### 预测序列
在做预测时我们只需要给定时间0的输入和起始隐含变量。然后我们每次将上一个时间的输出作为下一个时间的输入。
![image.png](http://zh.gluon.ai/_images/rnn_3.png)


### 训练模型
下面我们可以还是训练模型。跟前面前置网络的教程比，这里有以下几个不同。

- 通常我们使用困惑度（Perplexity）这个指标。
- 在更新前我们对梯度做剪裁。
- 在训练模型时，对时序数据采用不同批量采样方法将导致隐含变量初始化的不同。

### 困惑度（Perplexity）

回忆以下我们之前介绍的交叉熵损失函数。在语言模型中，该损失函数即被预测字符的对数似然平均值的相反数：

$\text{loss} = -\frac{1}{N} \sum_{i=1}^N \log p_{\text{target}_i}$

其中$N$是预测的字符总数，$p_{\text{target}_i}$是在第$i$个预测中真实的下个字符被预测的概率。

而这里的困惑度可以简单的认为就是对交叉熵做$\exp$运算使得数值更好读。

为了解释困惑度的意义，我们先考虑一个完美结果：模型总是把真实的下个字符的概率预测为1。也就是说，对任意的i来说，$p_{\text{target}_i}$。这种完美情况下，困惑度值为1。

我们再考虑一个基线结果：给定不重复的字符集合W及其字符总数$|W|$，模型总是预测下个字符为集合$W$中任一字符的概率都相同。也就是说，对任意的i来说，$p_{\text{target}_i}=1/|W|$。这种基线情况下，困惑度值为$|W|$。

最后，我们可以考虑一个最坏结果：模型总是把真实的下个字符的概率预测为0。也就是说，对任意的i来说，$p_{\text{target}_i}=0$。这种最坏情况下，困惑度值为正无穷。

任何一个有效模型的困惑度值必须小于预测集中元素的数量。在本例中，困惑度必须小于字典中的字符数$|W|$。如果一个模型可以取得较低的困惑度的值（更靠近1），通常情况下，该模型预测更加准确。

In [26]:
slim = tf.contrib.slim

learning_rate = 1e-1
max_steps = 10000
batch_size = 32
train_loss = 0.0
train_acc = 0.0
is_random_iter=True
epochs=200
num_steps=35 
learning_rate=1e-2
batch_size=32
is_lstm = False

#训练
print hidden_dim
num_inputs = num_outputs = vocab_size
input_placeholder = tf.placeholder(tf.float32, [num_steps, None, num_inputs])
state_h_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])
print 'here'

state_c_placeholder = tf.placeholder(tf.float32, [None, hidden_dim])
gt_placeholder = tf.placeholder(tf.int64, [num_steps, None, 1])

if is_lstm:
    # 当RNN使用LSTM时才会用到，这里可以忽略。
    outputs, state_h, state_c = rnn(input_placeholder, state_h_placeholder, state_c, params)
else:
    outputs, state_h = rnn(input_placeholder, state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)

loss = tf.losses.sparse_softmax_cross_entropy(logits=outputs,  labels=tf.reshape(gt_placeholder, (num_steps*batch_size, 1)))

var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name
op = tf.train.AdamOptimizer(learning_rate)

gradients = tf.gradients(loss, params)

#process gradients
clipped_gradients, norm = tf.clip_by_global_norm(gradients, 1)

train_op = op.apply_gradients(zip(clipped_gradients, params))

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

if is_random_iter:
    data_iter = data_iter_random
else:
    data_iter = data_iter_consecutive
for e in range(0, epochs):
    # 如使用相邻批量采样，在同一个epoch中，隐含变量只需要在该epoch开始的时候初始化。
    if not is_random_iter:
        state_h_init = np.zeros(shape=(batch_size, hidden_dim))
        if is_lstm:
            # 当RNN使用LSTM时才会用到，这里可以忽略。
            state_c_init = np.zeros((batch_size, hidden_dim))
    train_loss, num_examples = 0, 0

    for data, label in data_iter(corpus_indices, batch_size, num_steps):
        # 如使用随机批量采样，处理每个随机小批量前都需要初始化隐含变量。
        if is_random_iter:
            state_h_init = np.zeros(shape=(batch_size, hidden_dim))
            if is_lstm:
                # 当RNN使用LSTM时才会用到，这里可以忽略。
                state_c_init = np.zeros((batch_size, hidden_dim))


        feed_dict = {input_placeholder: get_inputs(data), state_h_placeholder: state_h_init, gt_placeholder: np.expand_dims(label.T, axis=-1)}
        loss_, state_h_, _ = sess.run([loss, state_h, train_op], feed_dict=feed_dict)
        state_h_init = state_h_

        print np.exp(loss_)


256
here
> <ipython-input-13-8aa99e2fec76>(8)rnn()
-> W_xh, b_h, W_hh, W_hy, b_y = params
(Pdb) c
rnn/weights_hidden
rnn/bias_hidden
rnn/times_hidden2
rnn/weights_output
rnn/bias_output
2613.1157
2562.6064
1356.2452
836.8392
713.81946
990.6877
910.2474
847.0908
855.73706
934.7522
861.5633
1063.3998
832.9008
1083.1605
857.855
861.56537
893.08905
644.0988
707.4264
773.52875
681.5286
707.54785
677.3389
776.0865
806.62024
861.33124
663.0542
705.9363
766.5731
679.5278
613.66626
905.4453
700.9081
688.0432
705.19006
786.2685
718.99207
928.6117
734.2491
637.65265
718.86865
701.18585
930.9748
803.7583
663.68176
748.9508
680.92865
750.66876
763.0859
676.98376
752.4348
888.2314
754.46906
807.6305
860.73096
692.7802
654.3308
622.24677
655.2203
815.702
652.4098
577.4837
657.4521
621.29443
679.71375
629.5921
744.3385
716.81934
763.11316
870.08624
814.01105
656.8385
733.24005
922.93915
695.4894
633.72455
631.2575
531.245
961.12683
627.05475
694.11945
750.67596
671.5402
677.8194
800.9944
790.1578
719.

144.62015
120.91494
128.24034
160.16138
157.78586
137.82658
194.49208
148.10132
118.167076
131.63533
149.37782
133.72632
163.39824
206.70755
174.1592
156.94356
93.65466
145.87102
107.19701
167.58984
130.22064
138.93103
176.17894
160.12823
138.76584
126.03417
177.2305
133.65964
117.295074
125.07633
142.55267
137.64368
155.26993
132.29346
129.59192
150.47131
149.43068
116.36919
116.84392
112.092255
116.15503
107.78054
118.19023
141.706
171.82545
123.28937
168.64891
113.65891
98.25254
112.78313
119.625114
106.39378
115.97245
109.73482
132.79338
155.03267
132.10378
130.95305
136.60483
115.32999
133.67819
120.62815
122.24654
119.19221
147.01376
142.2999
127.85592
129.11728
146.94942
132.58997
124.58953
130.19885
131.80383
126.11365
124.738205
97.500435
101.630684
136.68114
120.31456
97.48937
102.95727
112.14186
106.059525
110.783165
94.85397
99.234245
119.23342
77.24771
126.21731
119.26197
144.91476
129.86
117.34308
81.1937
92.52361
121.253746
137.22571
109.176414
101.750404
94.81228
102.85

17.486347
23.18891
23.284365
18.0588
18.31557
24.75195
22.028444
19.300673
23.9898
23.02223
21.715729
30.294954
24.040817
23.98518
24.850088
20.40987
21.9052
24.901787
33.1385
25.137175
17.996336
18.896572
18.331255
22.632824
20.980234
19.948217
19.63136
16.464512
20.696215
19.84313
24.616356
18.236124
22.784996
25.975357
22.633352
16.94075
22.158747
20.051609
21.633717
22.205996
16.594923
21.413454
21.714346
22.782719
25.017538
27.85674
26.850025
28.656794
23.41973
21.52245
23.875042
19.651505
17.528673
22.441408
20.21363
23.542606
20.547087
19.570095
22.937714
18.366892
26.899485
24.590082
19.597437
20.25116
23.977936
20.608978
26.189585
27.680939
30.460463
24.25601
24.568071
19.5905
23.443018
22.94902
23.238234
20.585161
17.804016
20.347185
20.466692
18.270874
16.17139
16.217352
21.371626
18.940853
19.411772
17.021385
18.383999
15.899582
22.133797
18.660065
15.721804
16.298857
21.610947
21.19359
21.193241
18.340723
17.178347
22.725708
21.310133
21.559582
18.471668
19.44734
16.6636
1

10.888358
10.352175
9.904557
12.6527605
12.533902
14.2910185
14.267994
12.0988245
18.218924
14.696936
12.173142
11.424559
12.975889
10.788124
13.16593
10.219822
13.038586
16.035152
14.113138
14.006725
14.756428
13.639251
13.705169
12.5035715
12.485766
12.003128
15.747122
13.641157
11.514859
12.445801
17.931746
14.938134
11.114203
12.031229
13.789614
11.354223
14.54688
11.410686
11.4355755
13.800602
11.85701
11.544767
10.347888
10.475391
10.90349
9.106583
12.112424
15.415691
10.122789
11.136256
10.975247
11.286739
11.857286
11.139944
12.455211
11.899458
12.363686
8.857467
10.153874
12.488136
14.054574
11.156937
10.92378
12.713274
12.272134
11.58677
10.852176
9.38623
9.575557
11.870211
12.315699
13.0829525
11.388907
12.20158
14.830594
14.975416
11.642399
18.156918
11.671107
9.906605
15.387707
9.489244
15.756781
13.439096
12.423533
16.422619
17.739426
11.547909
10.07371
10.806872
11.236367
11.339882
13.343405
15.66141
13.30963
11.534506
15.469854
17.305336
16.430166
9.982727
9.555199
10.6

7.895967
9.576739
10.890214
7.912202
9.774799
7.7730527
10.652109
9.040376
8.22997
7.7117977
9.129759
10.258661
7.586049
8.79496
11.005
12.945251
8.773286
10.041569
8.371642
7.947498
10.031947
9.241267
8.950905
10.353249
8.046792
12.707322
8.743803
9.954528
8.764405
8.460495
9.577699
10.242849
7.3799
8.905954
7.9521
7.536438
9.841569
9.5731945
10.768517
10.960996
8.205041
8.001661
9.178373
11.381774
12.062616
9.397998
8.090633
6.2115083
10.751752
8.872065
5.885891
8.016787
7.7734976
10.310416
7.0083885
10.788715
8.020982
10.635602
8.369204
9.572777
9.922709
7.7912507
9.339034
9.012979
8.336677
8.6468725
7.9695654
7.2515545
9.622209
10.591804
12.577541
9.700692
9.91597
7.661092
10.288468
10.324949
9.716968
8.685612
8.732624
8.54452
10.037019
11.357455
9.472563
8.262766
8.7362385
11.103831
9.771288
10.949336
8.610355
11.269093
10.411416
8.12723
8.678582
13.174726
11.855283
11.270503
9.788594
9.106192
9.737678
7.9531274
10.454866
8.203265
9.070305
8.647778
7.829318
7.632698
7.863483
8.149

...

5.999519
6.763575
5.8193474
7.379445
5.276352
7.468864
5.950689
8.195223
5.2653537
6.279234
8.00862
7.5872374
7.3439517
6.550315
9.108858
6.101895
7.1020417
7.1014986
8.15129
8.852748
5.9407406
7.608606
5.268203
6.894047
8.291709
6.515329
4.797822
5.3651705
5.689858
5.3818083
6.111181
4.639868
5.5251055
5.148104
5.646344
5.464289
5.053882
4.9364605
5.1719255
4.6057305
6.2229214
6.8747025
4.9038677
7.1652465
5.5000916
5.986245
6.117875
5.109546
5.5092306
5.82961
6.2048817
6.485304
6.9524803
5.9396296
4.591463
6.83346
4.469205
6.112517
6.5658774
5.20478
6.76342
5.707321
6.241008
6.7124853
6.9165993
6.903049
6.896601
6.0929165
6.3368587
5.454184
5.293538
7.8593783
6.362007
4.7768373
5.9884715
5.309442
7.3225136
5.319145
6.0683074
6.9707437
5.7624354
7.067576
5.5894337
5.4744606
4.5172434
6.6245193
5.3928185
4.9499674
5.210904
5.7707214
6.5738006
5.2613244
5.39714
6.379817
6.4605675
5.511274
6.310088
7.9081526
5.48461
5.409306
5.8298016
6.3653507
5.750739
5.47387
6.8462124
4.7729735
6.3637

5.222023
5.829663
6.2041664
6.50995
6.533112
5.8781943
6.2047586
8.550275
5.7529874
5.091765
5.3088093
4.110018
6.070492
4.8326335
5.024435
5.5389905
6.1049776
6.134387
5.0288043
5.5698233
5.6944304
5.9454274
5.4628367
5.592317
5.2569647
6.874413
5.884742
4.843015
4.816111
6.7556605
7.061084
6.522178
6.4552174
5.065318
7.237939
6.2686844
5.2941566
5.70561
5.944967
6.3707614
6.6439943
5.331785
5.853539
5.383374
6.074905
4.9491973
6.6892443
4.5079575
7.3952336
7.3111467
6.0133543
6.3546367
6.942132
6.800604
4.804001
8.573088
5.6221633
7.126985
6.4283257
8.060862
9.572263
6.3355885
5.717405
6.6521916
5.2114663
6.1591206
6.383999
5.8217893
5.3413587
5.307026
5.200085
5.624392
5.9099894
5.080809
5.255433
5.175727
6.910728
5.5295277
5.792439
6.6098614
5.420614
4.849419
5.765871
5.8261166
7.4986863
6.1527166
7.778861
7.1057463
6.003591
4.7494006
5.4600906
5.7777357
8.324797
6.214559
5.7460318
6.916919
5.526916
6.2532735
6.359035
6.1765766
6.329048
6.01044
6.009452
5.397324
4.923834
6.668344
6

5.365419
6.758963
6.9180164
5.1570797
6.348795
4.6604
4.4697394
5.57206
5.698286
4.980473
5.1757936
4.6027985
6.9011235
4.8877077
4.7855663
4.4733367
5.265545
4.1352496
6.5478854
5.4340634
5.550657
6.269579
5.7460704
6.6785936
7.6848187
6.945097
7.8337545
5.114797
7.135577
7.195028
5.6834507
6.2769623
5.5204105
5.5849233
6.975681
7.71685
4.315012
5.4365344
7.4456544
5.5163784
7.5365133
6.29452
7.260311
6.4033113
5.3498755
6.8261104
5.0569324
6.1977944
6.994195
5.7347293
4.763886
6.7966704
6.270557
7.5155535
5.1802063
5.203693
5.375694
5.403978
5.4340963
5.451756
5.837518
5.3908825
5.6401725
5.2681823
5.3093743
5.4875655
4.9408236
5.430591
5.455295
5.7954535
6.8882976
5.939445
5.600979
5.952136
5.295812
4.8352094
5.386702
6.004117
5.5596614
7.3716335
7.0167866
5.2018933
6.7362175
5.9689302
7.3339458
5.375795
5.0283785
5.552095
5.3319488
6.6144347
6.8735757
9.073853
5.4681835
5.468737
6.92355
6.3674207
6.259843
7.237021
6.356197
6.2807403
7.7264214
6.0419126
5.4112954
8.153751
5.0146832


6.6123037
5.2998996
4.858417
5.8752913
16.125877
6.7225747
5.012807
6.3625293
5.9764094
5.70121
4.8403955
5.818447
5.008959
6.012103
6.236434
5.6874366
6.5879726
4.902343
4.695232
6.1271834
6.5687184
6.5194125
5.204775
4.75002
6.2545357
6.1137395
6.60169
7.525846
6.9907656
5.319467
5.7253256
5.469204
6.277466
5.5349197
5.2819324
4.250081
6.337064
5.4160886
6.696577
6.083747
5.795418
6.6348433
8.806158
5.8233094
5.24166
5.1681848
17.359015
5.7114506
5.8168554
6.221369
5.290845
5.11456
6.692647
4.818975
7.5477
5.2781253
4.6945996
5.640835
6.419639
6.270439
5.3794165
5.3974676
6.371403
8.082902
5.3856864
5.3938723
4.747696
4.935952
4.9128423
5.4897017
5.4611278
6.662599
6.0346713
6.2153754
5.842995
4.977676
5.777966
5.4253244
5.9601665
5.790843
5.3695297
5.980664
4.9731326
6.5430264
6.5189548
6.119482
6.0735774
6.270926
5.512623
6.3004503
7.0316195
6.812288
4.6610775
7.091654
5.1114516
21.408892
5.6643453
7.350964
7.6617713
5.6729603
8.034534
6.499821
4.5607176
4.7879467
4.579649
5.908627

5.2225385
5.6693935
4.3100834
6.1897535
4.001413
5.689068
5.3788676
6.7867613
6.4918833
5.994375
4.940184
6.395176
5.4813538
6.387793
5.503599
6.5116687
6.416626
5.9070654
4.5948443
5.737674
6.262317
5.986783
6.2704115
5.7569056
5.7085605
5.1807756
6.073734
5.5069065
4.875312
6.773346
5.6240807
7.7894006
4.835159
4.608439
6.4368005
6.6371183
4.9743743
4.409467
5.3393297
5.0400357
6.2017417
5.0108075
6.4488487
5.3517423
4.0764174
6.067938
6.3006725
5.4099183
5.583044
6.0354457
5.0094013
5.1548123
5.262538
5.935754
4.8294435
4.187065
4.6754766
6.0107565
5.813598
4.843283
5.28326
5.886054
5.8168006
5.3542366
6.470751
5.2746115
4.372448
6.610167
5.410594
6.763641
5.533104
6.494804
6.3590336
5.184841
5.3891892
6.245387
6.198917
5.3749857
5.239857
4.8925347
8.442614
6.409203
6.863974
5.1390758
7.223962
6.899866
8.014913
6.031188
6.0477934
5.3626795
4.8557262
5.405455
5.2626505
5.6635966
6.8780994
4.826323
5.9917874
6.148167
5.3990235
4.6805162
5.7837076
4.928871
6.2687445
5.852085
5.7677035


In [29]:
epochs = 200
num_steps = 35
learning_rate = 1e-4
batch_size = 32


seq1 = '分开'.decode('utf-8')
seq2 = '不分开'.decode('utf-8')
seq3 = '战争中部队'.decode('utf-8')
seqs = [seq1, seq2, seq3]
print seq1

分开


我们先采用随机批量采样实验循环神经网络谱写歌词。我们假定谱写歌词的前缀分别为“分开”、“不分开”和“战争中部队”

In [32]:
pred_len = 100
test_num_steps = 1
# 预测
num_inputs = num_outputs = vocab_size    
test_input_placeholder = tf.placeholder(tf.float32, [test_num_steps, None, num_inputs])
test_state_h_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])
test_state_c_placeholder = tf.placeholder(tf.float32, [test_num_steps, hidden_dim])

if is_lstm:
    # 当RNN使用LSTM时才会用到，这里可以忽略。
    outputs, state_h, state_c = rnn(test_input_placeholder, test_state_h_placeholder, test_state_c_placeholder, params)
else:
    outputs, state_h = rnn(test_input_placeholder, test_state_h_placeholder, params)

outputs = tf.concat(outputs, axis=0)


var_list = tf.trainable_variables()
for var in var_list:
    print var.op.name


for seq in seqs:
    prefix = seq

    prefix = prefix.lower()
    test_state_h_init = np.zeros((1, hidden_dim))
    if is_lstm:
        # 当RNN使用LSTM时才会用到，这里可以忽略。
        test_state_c_init = np.zeros((1, hidden_dim))
    output_seq = [char_to_idx[prefix[0]]]
    for i in range(pred_len + len(prefix)):
        X = np.array([output_seq[-1]])
        # 在序列中循环迭代隐含变量。
        if is_lstm:
            # 当RNN使用LSTM时才会用到，这里可以忽略。
            Y, state_h_, state_c_ = sess.run([outputs, state_h, state_c], feed_dict={test_input_placeholder: get_inputs(X), test_state_c_placeholder: test_state_c_init, test_state_h_placeholder: test_state_h_init})
        else:
            Y, state_h_ = sess.run([outputs, state_h], feed_dict={test_input_placeholder: get_inputs(X), test_state_h_placeholder: test_state_h_init})
        test_state_h_init = state_h_
        if i < len(prefix)-1:
            next_input = char_to_idx[prefix[i+1]]
        else:
            next_input = np.argmax(Y[0])
        output_seq.append(next_input)
    print ''.join([idx_to_char[i] for i in output_seq])

    print()

rnn/weights_hidden
rnn/bias_hidden
rnn/times_hidden2
rnn/weights_output
rnn/bias_output
分开始让啊 你听见不是因为包安斑鸠 热手刻 别人的坏的阻碍 就算放开 飞檐 我就是一切 一岸 旁边比宇宙去 想逃都靠去 因为捞鱼的蠢游戏我们占敌逼咒语啦～～～～～ oh~oh 随风这世界过去年代 或许我还是
()
不分开 我的世界 离变成些一路 生活习惯 硬足字铺著 是空不出就古存浮雕多的日  想象 我在等 我在我学会尖叫 马蹄声车存在被拉着 方向开始移动 我拿出城市车歌谣 千年恩怨 因为飞接我没有几人 斑驳的城堡 随
()
战争中部队　 江湖在何时变不见 我右拳 多朦朧.月前的白芽底 雨水 这个人宠坏 过去 为啥咪铁支路直直 火车叨位去 为啥咪铁支路直直 火车叨位去 为啥咪铁支路直直 火车叨位去 为啥咪铁支路直直 火车叨位去 为啥咪
()
